# Machine-learning Notebook (without Dask)
- Machine_Learning_vB2_20170802
- A cleaner version of Machine_Learning_vB in the same folder
- preceeded by feature creation notebooks

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import random
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import mean_squared_error

/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/justingosses/anaconda/envs/MannvilleDask/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.2
0.23.3


In [3]:
import os

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [5]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"
machine_learning_dir = "../Machine_Learning/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

----------

### Load dataframes for Machine-Learning via a HDF5 file

train_X
train_y
test_X
test_y

In [6]:
h5_to_load = 'df_all_Col_preSplit_wTrainTest_ClassBalanced_PreML.h5'
train_X = pd.read_hdf(machine_learning_dir+h5_to_load, 'train_X')
train_y = pd.read_hdf(machine_learning_dir+h5_to_load, 'train_y')
test_X = pd.read_hdf(machine_learning_dir+h5_to_load, 'test_X')
test_y = pd.read_hdf(machine_learning_dir+h5_to_load, 'test_y')

In [7]:
train_X.head()

,DPHI,GR,ILD,NPHI,McMurray_Base_Qual,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,WellThickness,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,0.227,102.473,0.000,0.460,1,25.0,359.66,3,1,210.058,396.102,0.0,246.5,102.473,102.473,102.473,102.473,102.4730,102.4730,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473000,102.4730,102.4730,102.473,102.473,102.473,102.473,102.473000,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.269,26.625,30.179,0.355,1,25.0,359.66,3,1,207.558,396.102,2.5,246.5,25.825,26.625,50.213,26.625,32.7680,26.6250,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625000,26.6250,26.6250,26.625,26.625,26.625,26.625,26.625000,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,26.625,20.262,30.179,30.370,30.179,27.1088,30.1790,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179000,30.1790,30.1790,30.179,30.179,30.179,30.179,30.179000,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179,30.179
2,0.339,31.562,21.793,0.428,1,25.0,359.66,3,1,205.058,396.102,5.0,246.5,23.605,31.562,49.258,31.562,34.0164,31.5620,31.5620,31.5620,22.700,31.562,60.528,31.562,36.187143,31.562000,31.5620,31.5620,31.562,31.562,31.562,31.562,31.562000,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,31.562,16.583,21.793,25.975,21.793,21.5708,21.7930,21.7930,21.7930,14.586,21.793,26.774,21.793,21.316286,21.793000,21.7930,21.7930,21.793,21.793,21.793,21.793,21.793000,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793,21.793
3,0.291,51.257,7.449,0.452,1,25.0,359.66,3,1,202.558,396.102,7.5,246.5,40.739,37.621,72.481,51.257,53.8586,43.6546,43.6546,53.8586,37.621,37.621,87.074,60.965,56.284000,47.374143,50.6518,63.1256,37.621,51.257,88.401,51.257,59.97963

--------------------

In [8]:
seed = 123

In [ ]:
# .values.ravel()
model = XGBClassifier(
    gamma=0, 
    reg_alpha=0.2, 
    max_depth=3, 
    subsample=0.8, 
    colsample_bytree= 0.8, 
    n_estimators= 300, 
    learning_rate= 0.03, 
    min_child_weight= 3,n_jobs=8)
model.fit(train_X,train_y)


In [ ]:
result = model.predict(test_X)
result

In [ ]:
test_y_indexValues = test_y.index.values
df_result = pd.DataFrame(result, index=test_y_indexValues, columns=['TopMcMr_Pick_pred'])
df_results_2 = pd.concat([test_y, df_result], axis=1)